### Modular GCD

- Given integers A, B, N, calculate $A^N + B^N$ and $|A - B|$. Assume that $\text{GCD}(0,a) = a$ for any positive integer $a$. Since this number could be large, take modulo $10^9 + 7$
    - $1 \le A,B,N \le 10^{12}$
    - $B \le A$

- Given the contraints, $A^N$ and $B^N$ can be ridiculously large. So we can't necessarily store $A^N + B^N$ to compute the GCD. This is where our modular arithmetic comes in
    - Recall that $A^N \mod M = ((A \mod M) * (A \mod M) * ...) \mod M$
    - Recall that $(A + B) \mod M = (A \mod M + B \mod M) \mod M$

- At first glance, you maybe tempted to simply compute the modulo exponentials for $A^N, B^N$, and find the gcd of that value and the $|A - B|$ term. 
    - This is wrong
    - Let $X = A^N + B^N$ and $Y = |A - B|$
    - $\text{GCD}(X, Y) \neq \text{GCD}(X \mod M, Y \mod M)$
        - If $X = 100, Y = 20$, then $\text{GCD}(100, 20) = 20$, but $\text{GCD}(100 \mod 10, 20 \mod 10) = 2$

- There isn't really a shortcut, we need to try out a full list of candidate GCDs 
    - Get a list of all candidate divisors $d$ of smaller term (most likely the smaller term is $|A - B|$) [$O(\sqrt{|A - B|})$]
    - For each divisor, compute $A^N \mod d$ and $B^N \mod d$ [$O(\log(N))$]
        - This can be done easily, because modular product is $A^N \mod d = (A \mod N * A \mod N * ...) \mod N$
        - If $(A^N \mod d + B^N \mod d) \mod d = 0$, then $d$ is a divisor of both $A^N + B^N$ and $|A - B|$
        - Since we want the greatest common divisor, iterate from largest to samllest in the divisor list

In [459]:
import numpy as np
import random

# A, B, N = np.random.randint(1, 1e12+1, 3)
A, B, N = random.randint(1, 1e2+1), random.randint(1, 1e2+1), random.randint(20, 30)
# A, B, N = 9, 1, 5
modulo = int(1e9 + 7)

def modular_exponential(base: int, exponent: int, modulo: int) -> int:
    result = 1
    # base = base % modulo
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulo
            exponent -= 1
        
        base = (base * base) % modulo
        exponent //= 2
    return result 

def gcd(A: int, B: int) -> int:
    assert A >= 0
    assert B >= 0

    if A == 0:
        return B
    elif B == 0:
        return A

    return gcd(B, A % B)    

def modular_gcd(A: int, B: int, N: int) -> int:
    modulo = 1e9 + 7
    
    abs_diff = abs(A-B)
    gcd = 1

    if abs_diff == 0:
        return (modular_exponential(base=A, exponent=N, modulo=modulo) + modular_exponential(base=B, exponent=N, modulo=modulo)) % modulo

    for candidate_divisor in range(1, int(abs_diff**0.5//1)+1):
        candidate_divisor_conjugate = abs_diff / candidate_divisor
        if abs_diff % candidate_divisor == 0:
            an_conj = modular_exponential(base=A, exponent=N, modulo=candidate_divisor_conjugate)
            bn_conj = modular_exponential(base=B, exponent=N, modulo=candidate_divisor_conjugate)

            an = modular_exponential(base=A, exponent=N, modulo=candidate_divisor)
            bn = modular_exponential(base=B, exponent=N, modulo=candidate_divisor)

            if (((an_conj % candidate_divisor_conjugate) + (bn_conj % candidate_divisor_conjugate)) % candidate_divisor_conjugate) == 0:
                gcd = max(gcd, candidate_divisor_conjugate)
            elif (((an % candidate_divisor) + (bn % candidate_divisor)) % candidate_divisor) == 0:
                gcd = max(gcd, candidate_divisor)
                
    return gcd

for _ in range(10_000):
    A, B, N = random.randint(1, 1e2+1), random.randint(1, 1e2+1), random.randint(20, 30)

    ## Special case, will not match actual modulo value
    if A == B:
        continue

    if modular_gcd(A, B, N) != gcd(pow(A,N) + pow(B,N), abs(A-B)):
        print(f"{A=}, {B=}, {N=}, {modular_gcd(A, B, N)=}, {gcd(pow(A,N) + pow(B,N), abs(A-B))=}")
        break

### 